In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, GPTNeoModel
import torch
import numpy as np

In [2]:
fw = load_dataset("HuggingFaceFW/fineweb-edu", name="CC-MAIN-2024-10", split="train", streaming=True)

In [3]:
n_stream = 2500
texts = []
for x in fw:
    texts.append(x["text"])
    if len(texts) > n_stream: break

print(texts[:10])

['– Computer viruses are parasitic programs which are able to replicate themselves, attach themselves to other executables in the computer, and perform some unwanted and often malicious actions. A virus is not able to spread itself to another computers, some user actions are needed for it to infect a new computer. Downloading and running software from untrusted sources, inserting an USB drive without a previous scan–remember always disable the AutoRun feature for the drives as CD-ROMs, DVD-ROMs– , downloading and running emails or IM attachments even from known persons, can put you in the nasty situation to have an infected computer. Always when you deal with these situations and to prevent computer infections, scan before to run.\nThe best scanners in my opinion are multi-engine online scanners like virustotal.com or novirusthanks.org. The links of these scanners and many more are on the home page.\nThe main three features of a virus are :\n– the replication mechanism search and find 

In [ ]:

def extract_embeddings(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=False)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    return outputs.hidden_states[-1].mean(axis=(0, 1))


# Load pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
model = GPTNeoModel.from_pretrained("EleutherAI/gpt-neo-125m")
model.eval()

embeddings = []
i = 0
for x in fw:
    print(i)
    embeddings.append(extract_embeddings(x["text"], model, tokenizer))
    if len(embeddings) > n_stream: break
    i += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:
def threshold(x, lambd, clamp=True):
    x_ = np.sign(x) * np.maximum(np.abs(x) - lambd, 0)
    if clamp:
        x_[x_ < 0] = 0
    return x_

def alpha_update(X, D, alpha, lambd, max_iter=20):
    L = np.linalg.norm(D, ord=2) ** 2
    a = np.zeros_like(alpha)
    a_ = np.copy(a)
    
    t = 1
    for _ in range(max_iter):
        a_prev = np.copy(a)
        a = threshold(a_ + (1 / L) * D.T @ (X - D @ a_), lambd / L)
        t_new = (1 + np.sqrt(1 + 4 * t ** 2)) / 2
        a_ = a + ((t - 1) / t_new) * (a - a_prev)
        t = t_new
    
    return a

def dictionary_update(X, a):
    return X @ np.linalg.pinv(a)

In [ ]:
K = 50
X = torch.stack(embeddings).numpy().T
N, D = embeddings.shape
Phi = np.random.randn(D, K)  # Initial dictionary
alpha = np.random.rand(K, N)  # Initial sparse codes
l = 0.5  # Regularization parameter

# Perform dictionary learning
for i in range(25):
    alpha = alpha_update(X, Phi, alpha, l)
    Phi = dictionary_update(X, alpha)
    print(i)

In [ ]:
top_10_indices = np.argsort(alpha[23, :])[-10:][::-1]
[texts[i] for i in top_10_indices]